In [1]:
import graphlab

In [2]:
image_train = graphlab.SFrame('../data/week_6_image_classifier/image_train_data/')
print type(image_train), image_train.shape
image_test = graphlab.SFrame('../data/week_6_image_classifier/image_test_data/')
print type(image_test), image_test.shape

This non-commercial license of GraphLab Create for academic use is assigned to bmatthewtaylor@gmail.com and will expire on October 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\bmt\AppData\Local\Temp\graphlab_server_1480398169.log.0


<class 'graphlab.data_structures.sframe.SFrame'> (2005, 5)
<class 'graphlab.data_structures.sframe.SFrame'> (4000, 5)


In [3]:
graphlab.canvas.set_target('ipynb')

Split the SFrame with the training data into 4 different SFrames. Each of these will contain data for 1 of the 4 categories above. Hint: if you use a logical filter to select the rows where the ‘label’ column equals ‘dog’, you can create an SFrame with only the data for images labeled ‘dog’.

In [4]:
print image_train.column_names()

['id', 'image', 'label', 'deep_features', 'image_array']


In [9]:
labels = set(image_train['label'])
print len(labels), type(labels), labels

4 <type 'set'> set(['automobile', 'dog', 'bird', 'cat'])


In [42]:
automobile_image_train = image_train.filter_by('automobile', 'label')
print automobile_image_train.shape
print automobile_image_train['label'].head(10)

(509, 5)
['automobile', 'automobile', 'automobile', 'automobile', 'automobile', 'automobile', 'automobile', 'automobile', 'automobile', 'automobile']


In [43]:
dog_image_train = image_train.filter_by('dog', 'label')
print dog_image_train.shape
print dog_image_train['label'].head(10)

(509, 5)
['dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog']


In [44]:
bird_image_train = image_train.filter_by('bird', 'label')
print bird_image_train.shape
print bird_image_train['label'].head(10)

(478, 5)
['bird', 'bird', 'bird', 'bird', 'bird', 'bird', 'bird', 'bird', 'bird', 'bird']


In [45]:
cat_image_train = image_train.filter_by('cat', 'label')
print cat_image_train.shape
print cat_image_train['label'].head(10)

(509, 5)
['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat']


In [46]:
result = image_train['label'].sketch_summary()
print type(result)
print result
#https://turi.com/products/create/docs/generated/graphlab.Sketch.html

<class 'graphlab.data_structures.sketch.Sketch'>

+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+




create a nearest neighbor model using the 'deep_features' as the features, but this time create one such model for each category, using the corresponding subset of the training_data. 

In [47]:
automobile_knn_model = graphlab.nearest_neighbors.create(automobile_image_train,features=['deep_features'], label='id')
print type(automobile_knn_model)

Starting brute force nearest neighbors model training.

<class 'graphlab.toolkits.nearest_neighbors._nearest_neighbors.NearestNeighborsModel'>


In [48]:
dog_knn_model = graphlab.nearest_neighbors.create(dog_image_train,features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

In [49]:
bird_knn_model = graphlab.nearest_neighbors.create(bird_image_train,features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

In [50]:
cat_knn_model = graphlab.nearest_neighbors.create(cat_image_train,features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

In [51]:
cat = image_test[0:1]
cat['image'].show()

What is the nearest ‘cat’ labeled image in the training data to the cat image above (the first image in the test data)?

In [70]:
result = cat_knn_model.query(cat)
print "result['distance'] = ", result['distance']
print "-------------"
print result['distance'].sketch_summary()


Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.008ms      |

| Done         |         | 100         | 36.034ms     |

+--------------+---------+-------------+--------------+

[34.62371920804245, 36.00687992842462, 36.52008134363789, 36.754850252057054, 36.87312281675268]
-------------

+--------------------+----------------+----------+
|        item        |     value      | is exact |
+--------------------+----------------+----------+
|       Length       |       5        |   Yes    |
|        Min         |  34.623719208  |   Yes    |
|        Max         | 36.8731228168  |   Yes    |
|        Mean        | 36.1557307098  |   Yes    |
|        Sum         | 180.778653549  |   Yes    |
|      Variance      | 0.675112568409 |   Yes    |
| Standard Deviation | 0.821652340354 |   Yes    |
|  # Missing Values  |       0        |   Yes    |
|  # unique values   |       5        |    No    |
+--------------------+----------------+----------+

Most frequent items:
+-------+---------------+---------------+---------------+---------------+--------------+
| value | 36.5200813436 | 36.7548502521 | 36.8731228168 | 36.0068799284 | 34.623719208 |
+-------+---------------+

In [ ]:
#mean of result['distance'] from cats = 36.1557307098


In [53]:
#function get_images_from_ids 
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [54]:
show_neighbors = lambda i: get_images_from_ids(cat_knn_model.query(image_train[i:i+1]))['image'].show()

In [55]:
show_neighbors(0)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.513ms      |

| Done         |         | 100         | 28.533ms     |

+--------------+---------+-------------+--------------+

In [56]:
cat_neighbors = get_images_from_ids(cat_knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 5.512ms      |

| Done         |         | 100         | 27.531ms     |

+--------------+---------+-------------+--------------+

In [57]:
cat_neighbors['image'].show()

In [58]:
dog_neighbors = get_images_from_ids(dog_knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 7.008ms      |

| Done         |         | 100         | 29.03ms      |

+--------------+---------+-------------+--------------+

In [59]:
dog_neighbors['image'].show()

In [71]:
result = dog_knn_model.query(cat)
print "result['distance'] = ", result['distance']
print "-------------"
print result['distance'].sketch_summary()


Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.01ms       |

| Done         |         | 100         | 29.031ms     |

result['distance'] =  

+--------------+---------+-------------+--------------+

[37.464262878423774, 37.56668321685285, 37.60472670789396, 37.70655851529755, 38.511325490739715]
-------------

+--------------------+----------------+----------+
|        item        |     value      | is exact |
+--------------------+----------------+----------+
|       Length       |       5        |   Yes    |
|        Min         | 37.4642628784  |   Yes    |
|        Max         | 38.5113254907  |   Yes    |
|        Mean        | 37.7707113618  |   Yes    |
|        Sum         | 188.853556809  |   Yes    |
|      Variance      | 0.143142787585 |   Yes    |
| Standard Deviation | 0.378342156764 |   Yes    |
|  # Missing Values  |       0        |   Yes    |
|  # unique values   |       5        |    No    |
+--------------------+----------------+----------+

Most frequent items:
+-------+---------------+---------------+---------------+---------------+---------------+
| value | 37.7065585153 | 37.5666832169 | 37.6047267079 | 37.4642628784 | 38.5113254907 |
+-------+-------------

In [72]:
#quiz q's 4-6
# mean of result['distance'] from dogs = 37.7707113618
# mean of result['distance'] from cats = 36.1557307098
# first image in the test data is closer to its 5 nearest neighbors in the ‘cat’ data than in the ‘dog’ data.
#Q's 4-6 submitted and correct!!!!

In [ ]:
#In what range is the accuracy of the 1-nearest neighbor classifier at classifying ‘dog’ images from the test set?
#predict all dog images using the dog classifier, then calc accuracy.
#predict all images using the dog classifier, then calc accuracy.
#predict all images using a general classifier, then calc accuracy.
#predict all dog images using a general classifier, then calc accuracy.
# see reading section for detailed description of task
#80-90% is wrong.